<a href="https://colab.research.google.com/github/codethechangehmc/red-cross/blob/main/zensvi_arushi_transform_download_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing zensvi.
benjidad Creator ID: 109259731308049

In [2]:
!pip install zensvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of timezonefinder to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of timezonefinder to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of transformers to determine which version is compatible 

To import images from the dataset, we specify the user who uploaded the images on Mapillary (benjidad) and make an API call that retrieves the IDs of all the images.

In [1]:
from zensvi.download import MLYDownloader
import requests
import csv

# ---------------- CONFIG ----------------
ACCESS_TOKEN = "PASTE API KEY"
USERNAME = "benjidad"
MAX_IMAGES = 100          # change this to fetch more/less
CSV_FILE = "mapillary_coords.csv"
# ---------------------------------------

url = f"https://graph.mapillary.com/images?access_token={ACCESS_TOKEN}&creator_username={USERNAME}&fields=id,computed_geometry&limit=100"

coords = []
count = 0

while url and count < MAX_IMAGES:
    r = requests.get(url)
    data = r.json()

    for img in data.get("data", []):
        geom = img.get("computed_geometry", {}).get("coordinates")
        if geom:
            lon, lat = geom
            coords.append([lat, lon])
            count += 1
            if count >= MAX_IMAGES:
                break

    # follow pagination
    url = data.get("paging", {}).get("next")

# save CSV
with open(CSV_FILE, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["latitude", "longitude"])
    writer.writerows(coords)

print(f"✅ Saved {len(coords)} points to {CSV_FILE}")

✅ Saved 100 points to mapillary_coords.csv


Download CSV file produced onto PC:

In [11]:
from google.colab import files
files.download("mapillary_coords.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Using CSV file and Zensvi to get images:


In [5]:
from zensvi.download import MLYDownloader

# with a csv file with lat and lon:
ACCESS_TOKEN = "PASTE API KEY"
downloader = MLYDownloader(mly_api_key=ACCESS_TOKEN)
downloader.download_svi("images", input_csv_file="/content/mapillary_coords.csv", buffer = 0.5)

Getting pids...


Loading cache files: 0it [00:00, ?it/s]


[Vector Tiles API] Fetching 2 tiles for images ...


Filtering data: 100%|██████████| 223014/223014 [10:49<00:00, 343.19it/s]


The panorama IDs have been saved to images/mly_pids.csv
The cache directory for tiles has been deleted


Getting urls by batch size 38: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it]

The cache directory has been deleted


In [6]:
import os

directory = "images/mly_svi/batch_1"
num_files = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
print(f"Number of files: {num_files}")

Number of files: 38


In [2]:
import shutil

folder = "/content/images"  # change to your folder path
shutil.rmtree(folder)  # deletes folder + all files/subfolders
print("Folder deleted!")

Folder deleted!


In [5]:
!pip uninstall -y cudf-polars-cu12 polars
!pip install polars==0.20.31


Found existing installation: cudf-polars-cu12 25.6.0
Uninstalling cudf-polars-cu12-25.6.0:
  Successfully uninstalled cudf-polars-cu12-25.6.0
Found existing installation: polars 1.25.2
Uninstalling polars-1.25.2:
  Successfully uninstalled polars-1.25.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 19.7 MB/s eta 0:00:00


In [7]:
!find . -name "polars.py"


In [7]:
from zensvi.metadata import MLYMetadata

path_input = "/content/images/mly_pids.csv"
mly_metadata = MLYMetadata(path_input)
mly_metadata.compute_metadata(
    unit="image", # unit of the metadata. Other options are "street" and "grid"
    indicator_list="all", # list of indicators to compute. You can specify a list of indicators in space-separated format, e.g., "year month day" or "all" to compute all indicators
    path_output="/content/output.csv" # path to the output file
)

/usr/local/lib/python3.12/dist-packages/zensvi/metadata/mly_metadata.py:234: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  self.street_network = ox.graph_from_bbox(
/usr/local/lib/python3.12/dist-packages/zensvi/metadata/mly_metadata.py:266: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  self.street_network = self.street_network.with_columns(


Computing timezones:   0%|          | 0/38 [00:00<?, ?it/s]

Computing datetimes:   0%|          | 0/38 [00:00<?, ?it/s]

,id,captured_at,compass_angle,creator_id,is_pano,sequence_id,lon,lat,organization_id,timezone,...,h3_6,h3_7,h3_8,h3_9,h3_10,h3_11,h3_12,h3_13,h3_14,h3_15
0,1083383342918549,1695713726000,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
1,1592197601515626,1695713726800,0.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
2,426745466675658,1695713727200,0.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
3,729073915997430,1695713727800,0.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
4,429854126132667,1695713728000,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
5,930691721881048,1695713728800,0.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
6,7286645284762958,1695713729000,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833555,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c58
7,1152211392455623,1695713729800,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833549,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c59
8,956015936149606,1695713730000,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833549,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c59
9,1910056746077348,1695713730800,270.000000,110051477895643,False,3VHwLztou8m4abd1hplCSg,106.833549,-6.249622,NaN,Asia/Jakarta,...,868c107afffffff,878c1078cffffff,888c107aa7fffff,898c107aa6fffff,8a8c107aa6d7fff,8b8c107aa6d3fff,8c8c107aa6d3dff,8d8c107aa6d3c7f,8e8c107aa6d3c5f,8f8c107aa6d3c59


Playing w/ Augmentation Techniques:

In [11]:
from zensvi.transform import ImageTransformer

dir_input = "/content/images/mly_svi/batch_1"
dir_output = "/content/images/mly_svi/transform3"
image_transformer = ImageTransformer(
    dir_input=dir_input,
    dir_output=dir_output
)
image_transformer.transform_images(
    style_list="perspective",  # list of projection styles in the form of a string separated by a space
    FOV=120,  # field of view
    theta=120,  # angle of view (horizontal)
    phi=0,  # angle of view (vertical)
    aspects=(9, 16),  # aspect ratio
    show_size=100,  # size of the image to show (i.e. scale factor)
    use_upper_half=False,  # use the upper half of the image for sky view factor calculation
)

Converting to perspective: 100%|██████████| 38/38 [00:34<00:00,  1.12it/s]
